In [ ]:
import numpy as np
import time
natom = 20899
nf = 20000
skip = 5000
nbin = 520
n_Na = 6145

box = np.array([47, 47, 100])
A = box[0] * box[1]
V = A * box[2] #* 6.022 * 1e-4
rho = n_Na / V
print(rho)
#print("A:", A, "V:", V, "rho:", rho)

dr = box[2] / nbin
#print("dr:", dr)

Na_density = np.zeros((nbin, nf - skip))
DATA_PATH=f'/Volumes/Tony/electrode_electrolyte/uranyl_nitrate_graphene/0.5M/330K/with_electrode/0V/eq'
name=f'unwg_eq_0V'
with open(f'{DATA_PATH}/{name}.lammpstrj', 'r') as input_file:
    for _ in range(skip):
        for _ in range(natom + 9):
            next(input_file)

    tps = 0
    eta = ""
    t_start = time.perf_counter()
    for i in range(nf - skip):
        print(f"\rts = {i}, tps = {tps:0.2f}, eta = {eta}", end="\r")
        for _ in range(9):
            next(input_file)

        r = np.zeros((n_Na, 3))
        n = 0

        for j in range(natom):
            line = next(input_file).split()
            id, atype, mol, q, x, y, z = map(float, line)
            if atype == 5:
                n += 1
                r[n - 1, 0] = x
                r[n - 1, 1] = y
                r[n - 1, 2] = z
        for m in range(nbin):
            d = 0+ m * dr
            count_Na=0
            for j in range(n_Na):
                if r[j,2]<d and r[j,2]>d-dr:
                    count_Na+=1
            Na_density[m, i] = count_Na / ((A * dr)*rho)
            t_end = time.perf_counter()
            tps = (t_end - t_start) / (i + 1)
            m, s = divmod(int(tps * ((nf-skip) - i - 1)), 60)
            h, m = divmod(m, 60)
            eta = f"{h:d}:{m:02d}:{s:02d}"
with open(f'density/{name}_water_330K.txt', 'w') as output_file:
    for m in range(nbin):
        d = 0 + m * dr
        sumNa = np.mean(Na_density[m, :])
        output_file.write(f"{d + (dr / 2.0)} {sumNa}\n")